In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import drive
import os
from huggingface_hub import notebook_login
notebook_login()
# os.makedirs("/content/drive/MyDrive/data/tokenized_data")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets
dataset = load_dataset(
    "coded-by-49/Langbridge_wazobia_data",
    split="train",
    streaming=True  # enables line-by-line streaming
)

for index, sample in enumerate(dataset):
  sample['translation'] = { k:v for k, v in sample['translation'].items() if v}
  print(sample)
  if index == 4:
    break

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

{'translation': {'eng_Latn': 'The way of life that he recommends “ holds promise of the life now and that which is to come . ”', 'ibo_Latn': 'Ụzọ ndụ ọ chọrọ ka anyị gbasoo “ nwere nkwa maka ndụ dị ugbu a na nke ahụ gaje ịbịa . ”'}}
{'translation': {'eng_Latn': 'No indeed ; soon will they come to know ;', 'hau_Latn': "A 'aha ! Zã su sani ."}}
{'translation': {'eng_Latn': "Our burning desire to help people get to know God is a reflection of Jehovah 's zeal .", 'hau_Latn': 'Kasancewa da himma wajen taimaka wa mutane su san Allah ya nuna cewa muna yin koyi da Jehobah .'}}
{'translation': {'yor_Latn': 'Ṣé ò ń yẹra fún ẹ̀mí yìí ?', 'eng_Latn': 'Do you resist it ?'}}
{'translation': {'yor_Latn': 'Pẹ̀lú ilé-ìwòsàn 14 àti ilé-ìwòsàn alábọ́dé 45 tí ó ti di títì pa, àwọn obìnrin kan ní láti bímọ sí abẹ́ ahéré, ilé wọn, ní àdúgbò láàrin àwọn ọ̀nà tí kò dára, tàbí ní ile ìwòsàn tí ó kún àkúnfàya, níbi tí ọ̀rọ̀ ìmọ́tótó ti ń burú sí i, àti pé ewu àti kó àrùn àti ìpalára ètò ìlera ń pọ̀ sí i.', 'eng

## **Data tokenization**

In [ ]:
import torch
from itertools import islice
from transformers import (
    AutoTokenizer,
)
from peft import LoraConfig, PeftModel, get_peft_model, TaskType, LoftQConfig
# from trl import SFTTrainer
import json
import pyarrow as pa
import pyarrow.ipc as ipc
from datasets import load_dataset, Dataset, concatenate_datasets

# defining the structure of all my arrow files
# schema = pa.schema([
#     ("input_ids", pa.list_(pa.int64())),
#     ("attention_mask", pa.list_(pa.int64())),
#     ("labels", pa.list_(pa.int64())),
# ])

# "facebook/nllb-200-distilled-600M"
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def process_data_iteratively(language_pairs, buffer_size):
  unprocessed_data_buffer = []
  processed_data_storagepath= "/content/drive/MyDrive/data/tokenized_data/igbo_test_val"
  successfullyprocessed_data = 0
  unproccessed_data = 0
  count = 0
  number_of_path_used = 1
  # streamed_dataset = load_dataset("coded-by-49/Langbridge_wazobia_data", streaming=True, split="train")

  train_url = "https://huggingface.co/datasets/coded-by-49/Langbridge_wazobia_data/resolve/main/wazobia_data_train_only.jsonl"
  hau_test_url = "https://huggingface.co/datasets/coded-by-49/Langbridge_wazobia_data/resolve/main/hau_test_val.jsonl"
  yor_test_url = "https://huggingface.co/datasets/coded-by-49/Langbridge_wazobia_data/resolve/main/yor_test_val.jsonl"
  igbo_test_url = "https://huggingface.co/datasets/coded-by-49/Langbridge_wazobia_data/resolve/main/igbo_test_val.jsonl"

  streamed_dataset = load_dataset("json", data_files=igbo_test_url, streaming=True)

  next_line = iter(streamed_dataset['train'])
  done_processing = False;
  while not done_processing:
    while True:
      if(len(unprocessed_data_buffer)<= buffer_size-1):
        try:
          unprocessed_data_buffer.append(next(next_line))
        except StopIteration:
          count += 1
          print(f"The {count} batch of the dataset with a size of {len(unprocessed_data_buffer)} have been successfully gathered.")
          done_processing = True;
          break
      else:
        count += 1
        break

    print(f"The {count} batch of the dataset with a size of {len(unprocessed_data_buffer)} have been successfully gathered.")
    lang_map = {source: target for source, target in language_pairs}
    processed_data_buffer = []
    dataset = None

    for i,sample in enumerate(unprocessed_data_buffer):
        source_lang = None
        for lang_code in lang_map.keys():
          if lang_code in sample["translation"].keys() and sample['translation'][lang_code] != None :
              source_lang = lang_code
        if not source_lang:
            unproccessed_data += 1
            continue
        if lang_map[source_lang] != "eng_Latn":
            unproccessed_data += 1
            continue
        target_lang =  lang_map[source_lang]

        tokenizer.src_lang = source_lang
        tokenizer.tgt_lang = target_lang

        source_text = sample["translation"].get(source_lang)
        target_text = sample["translation"].get(target_lang)

        model_inputs = tokenizer(text = source_text, text_target = target_text, max_length=128, truncation=True)
        processed_data_buffer.append(model_inputs)
    dataset = Dataset.from_list(processed_data_buffer)

    try:
      if len(dataset) == len(processed_data_buffer):
        successfullyprocessed_data += len(dataset)
        dataset.save_to_disk(processed_data_storagepath)
        print(f"Successfully processed {len(dataset)} into {processed_data_storagepath}")
        processed_data_buffer.clear()
        unprocessed_data_buffer.clear()
        number_of_path_used +=1
        processed_data_storagepath = "/content/drive/MyDrive/data/tokenized_data/igbo_test_val/"[:-1] + str(number_of_path_used) + "/"
        print(f"the name of the next path is {processed_data_storagepath}")
      else:
        print(f"There is a discorcodance between the data being loaded and the data that was processed")
    except Exception as e:
      print(f"An error occurred: {e}")
  print(f"Succesfully processed data {successfullyprocessed_data} \n Flawed/unsuccessfull data was {unproccessed_data}")

language_pairs = [("ibo_Latn","eng_Latn"), ("hau_Latn","eng_Latn"), ("yor_Latn","eng_Latn")]
process_data_iteratively(language_pairs,200000)

The 1 batch of the dataset with a size of 93319 have been successfully gathered.
The 1 batch of the dataset with a size of 93319 have been successfully gathered.


Saving the dataset (0/1 shards):   0%|          | 0/93319 [00:00<?, ? examples/s]

Successfully processed 93319 into /content/drive/MyDrive/data/tokenized_data/igbo_test_val
the name of the next path is /content/drive/MyDrive/data/tokenized_data/igbo_test_val2/
Succesfully processed data 93319 
 Flawed/unsuccessfull data was 0


**Saving data into shards for model training**



In [ ]:
import os
from datasets import load_from_disk, concatenate_datasets

base_path = "/content/drive/MyDrive/data/"
dataset_paths = [os.path.join(base_path, "tokenized_data")]

for i in range(2, 16): # Loops from 2 to 15
    path = os.path.join(base_path, f"tokenized_data{i}")
    if os.path.exists(path):
        dataset_paths.append(path)
    else:
        print(f"Warning: Path not found - {path}")

print(f"Found {len(dataset_paths)} dataset shards to merge.")

print(dataset_paths)

try:
    loaded_datasets = [load_from_disk(path) for path in dataset_paths]

    # Merge them into one big dataset
    full_train_dataset = concatenate_datasets(loaded_datasets)
    print(f"Merge Successful! Total Training Examples: {len(full_train_dataset)}")

    full_train_dataset = full_train_dataset.shuffle(seed=42)

    # 5. Save the unified dataset
    save_path = os.path.join(base_path, "NLLB_FINAL_TRAIN_DATA")
    print(f"Saving merged dataset to: {save_path} ...")

    full_train_dataset.save_to_disk(save_path)
    print("Process Complete. You can now delete the old shards to save space if needed.")

except Exception as e:
    print(f"An error occurred: {e}")

Found 15 dataset shards to merge.
['/content/drive/MyDrive/data/tokenized_data', '/content/drive/MyDrive/data/tokenized_data2', '/content/drive/MyDrive/data/tokenized_data3', '/content/drive/MyDrive/data/tokenized_data4', '/content/drive/MyDrive/data/tokenized_data5', '/content/drive/MyDrive/data/tokenized_data6', '/content/drive/MyDrive/data/tokenized_data7', '/content/drive/MyDrive/data/tokenized_data8', '/content/drive/MyDrive/data/tokenized_data9', '/content/drive/MyDrive/data/tokenized_data10', '/content/drive/MyDrive/data/tokenized_data11', '/content/drive/MyDrive/data/tokenized_data12', '/content/drive/MyDrive/data/tokenized_data13', '/content/drive/MyDrive/data/tokenized_data14', '/content/drive/MyDrive/data/tokenized_data15']
Merge Successful! Total Training Examples: 3000000
Saving merged dataset to: /content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA ...


Saving the dataset (0/2 shards):   0%|          | 0/3000000 [00:00<?, ? examples/s]

Process Complete. You can now delete the old shards to save space if needed.


In [ ]:
!zip -r  "/content/drive/MyDrive/data/nllb-training-data-merged.zip" "/content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA"

updating: content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA/ (stored 0%)
updating: content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA/data-00000-of-00002.arrow (deflated 68%)
updating: content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA/data-00001-of-00002.arrow (deflated 68%)
updating: content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA/state.json (deflated 48%)
updating: content/drive/MyDrive/data/NLLB_FINAL_TRAIN_DATA/dataset_info.json (deflated 69%)


In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk

path = "/content/drive/MyDrive/data/tokenized_data/igbo_test_val"
# Use load_from_disk for datasets saved as directories
steam_dataset = load_from_disk(path)
print(steam_dataset[0])

{'input_ids': [256073, 10866, 792, 12230, 62, 26, 6628, 2540, 2819, 245, 2347, 17387, 147, 19402, 85913, 112, 248259, 13933, 3167, 248070, 10723, 147559, 9, 98957, 5, 3, 33962, 4001, 2347, 2540, 362, 248105, 138987, 14259, 258, 160, 248079, 2819, 2347, 49327, 2347, 62, 792, 2897, 24324, 62, 9, 133461, 188, 160, 248079, 99, 792, 5056, 62, 792, 53962, 53962, 99, 9, 55, 5681, 160, 248079, 792, 42429, 205599, 6782, 71, 99, 31234, 6782, 12312, 160, 189, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [256047, 57005, 575, 3277, 10275, 7131, 2442, 4414, 242693, 618, 11335, 37, 10771, 248079, 62354, 5884, 2442, 34253, 452, 349, 122898, 618, 1913, 144253, 248079, 349, 53481, 248, 1482, 3203, 5884, 29401, 5924, 248079, 796, 34796, 5884, 37517, 108, 5924, 248079, 540, 3203, 5884, 37517,

### Testing of Data collator for Transformer traning 
 

In [ ]:
from transformers import DataCollatorForSeq2Seq

# 1. Initialize the Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,    
    model=model_name,            
    padding=True,           
    label_pad_token_id=-100 
)

example_short = {
    'input_ids': [256073, 10866, 792, 12230, 2],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [256047, 57005, 575, 2]
}

example_long = {
    'input_ids': [256073, 10866, 792, 12230, 2, 12230, 12230, 12230 ],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
    'labels': [256047, 57005, 575, 2, 3, 5533, 3535, 535352]
}

batch_input = [example_long, example_short]

batch_output = data_collator(batch_input)


print("\n--- RESULTS ---")
print(f"Batch Input Keys: {batch_output.keys()}")
print(f"Tensor Shape: {batch_output['input_ids'].shape}")



--- RESULTS ---
Batch Input Keys: KeysView({'input_ids': tensor([[256073,  10866,    792,  12230,      2,  12230,  12230,  12230],
        [256073,  10866,    792,  12230,      2,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0]]), 'labels': tensor([[256047,  57005,    575,      2,      3,   5533,   3535, 535352],
        [256047,  57005,    575,      2,   -100,   -100,   -100,   -100]])})
Tensor Shape: torch.Size([2, 8])
